### Uses Paper Trade - test money
##### https://alpaca.markets/docs/api-documentation/api-v2/

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import time, datetime
import pandas as pd

import alpha, database
from populate import *
from portfolios import *
from state import *

DataStore = database.DataStore()

In [2]:
state = State(DataStore, 'AAPL', indicators=['ADX', 'CCI', 'EMA'])

Available to use:
  <classname>.prices
  <classname>.indicators


In [13]:
import gym
import numpy as np
from gym import spaces
import pandas as pd

class StockTrader(gym.Env):
    
    def __init__(self, dfPrices=None, dfIndicators=None):
        """dfIndicators is a DataFrame where each column is a different indicator"""
        self.indicators = dfIndicators
        self.prices = dfPrices
        assert self.indicators is not None and isinstance(self.indicators, pd.DataFrame), "need indicator dataframe"
        assert self.prices is not None and isinstance(self.prices, pd.DataFrame), "need prices dataframe"
        num_indicators = self.indicators.shape[1]
        assert num_indicators > 0, "supply 1 or more indicators"

        self.action_space = spaces.Discrete(3)

        # set space for alpha indicators at +- infinity...?
        low_array = np.full((num_indicators), -np.inf)
        high_array = np.full((num_indicators), np.inf)
        self.observation_space = spaces.Box(low=low_array, high=high_array, dtype=np.float64)
        
        self.previous_price = 0  # didn't exist before first day, so set previous_price to 0
        first_day = self.indicators.index[0] # starting at first day indicators exist
        self.prices = self.prices.loc[first_day:] # rewriting prices to fit indicator list
        
        self.state = np.array(self.indicators.iloc[0]) # first day is inititial state
        self.days = iter(self.prices.index.values)
        
        # Iterate through days, checking action/reward, etc. in step...

    def reset(self):
        return np.array(self.indicators.iloc[0])
    
    def step(self, action):
        #https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py
        err_msg = "%r (%s) invalid" % (action, type(action))
        assert self.action_space.contains(action), err_msg
        
        #self.state = 
        #return <obs>, <reward: float>, <done: bool>, <info: dict>

In [14]:
trader = StockTrader(state.prices, state.indicators)